## Implement unpadded message recovery oracle

Nate Lawson says we should stop calling it "RSA padding" and start calling it "RSA armoring". Here's why.

Imagine a web application, again with the Javascript encryption, taking RSA-encrypted messages which (again: Javascript) aren't padded before encryption at all.

You can submit an arbitrary RSA blob and the server will return plaintext. But you can't submit the same message twice: let's say the server keeps hashes of previous messages for some liveness interval, and that the message has an embedded timestamp:

```json
{
  time: 1356304276,
  social: '555-55-5555',
}
```

You'd like to capture other people's messages and use the server to decrypt them. But when you try, the server takes the hash of the ciphertext and uses it to reject the request. Any bit you flip in the ciphertext irrevocably scrambles the decryption.

This turns out to be trivially breakable: 

* Capture the ciphertext C
* Let N and E be the public modulus and exponent respectively
* Let S be a random number > 1 mod N. Doesn't matter what.
* Now:

```
C' = ((S**E mod N) C) mod N
```

* Submit C', which appears totally different from C, to the server, recovering P', which appears totally different from P
Now:

```
          P'
    P = -----  mod N
          S
```

 Oops!

Implement that attack.
### Careful about division in cyclic groups.
Remember: you don't simply divide mod N; you multiply by the multiplicative inverse mod N. So you'll need a modinv() function.




### Unpadded Message Oracle
Configure the server's RSA keypair and textbook RSA decrypt function.

In [1]:
from Crypto.Util.number import getPrime

In [2]:
NUM_BITS = 1024
p = getPrime(NUM_BITS)
q = getPrime(NUM_BITS)

In [3]:
n = p * q
n.bit_length()

2047

In [5]:
phi = (p - 1) * (q - 1) # Euler's totient.

In [7]:
d = pow(e, -1, phi)
d

9999084168040618735634861551028646714742419824776844939618046517930489960096612274105788204465626306989470696783698343279280014861995265035569539280891631064297939780160124628869695269343557975135982891073329303818009473603698633092090004042698768900183665105420632261310775144887374464668624168201333009266899950812475295837304653006366264235873345873988264006742306399022556182903044439835865635144643674405954517323848976805487216372176489900871366907034146005163955964589454433788255139460061307379840363093156441943600475313978033317975884382581433172679113666070300725777779503999992424431816057691268724076147

In [9]:
assert e * d % phi == 1

In [5]:
N = 14998626252060928103452292326542970072113629737165267409427069776895734940144918411158682306698439460484206045175547514918920022292992897553354308921337446596446909670240186943304542904015336962703974336609993955727014210405547949638135006064048153350275497658130948391966162717331061697002936252301999513900597824741726854402436970305499076189535485327631912218072518428911792985143753609764640004919486259062931216650284588585901673444541101407504191882322230182874555079901080636451642823862690883410812084807478665973587954142541349197747345526288869524033493400650480652831708745653869356203205959909973122017029
e = 3 # public exponent.

In [2]:
from Crypto.Util.number import bytes_to_long, long_to_bytes

In [3]:
m = b"retreat"

In [6]:
C = pow(bytes_to_long(m), e, N)

In [7]:
def decrypt(c):
    N = 14998626252060928103452292326542970072113629737165267409427069776895734940144918411158682306698439460484206045175547514918920022292992897553354308921337446596446909670240186943304542904015336962703974336609993955727014210405547949638135006064048153350275497658130948391966162717331061697002936252301999513900597824741726854402436970305499076189535485327631912218072518428911792985143753609764640004919486259062931216650284588585901673444541101407504191882322230182874555079901080636451642823862690883410812084807478665973587954142541349197747345526288869524033493400650480652831708745653869356203205959909973122017029
    d = 9999084168040618735634861551028646714742419824776844939618046517930489960096612274105788204465626306989470696783698343279280014861995265035569539280891631064297939780160124628869695269343557975135982891073329303818009473603698633092090004042698768900183665105420632261310775144887374464668624168201333009266899950812475295837304653006366264235873345873988264006742306399022556182903044439835865635144643674405954517323848976805487216372176489900871366907034146005163955964589454433788255139460061307379840363093156441943600475313978033317975884382581433172679113666070300725777779503999992424431816057691268724076147
    return long_to_bytes(pow(c, d, N))

Test the "oracle".

In [8]:
decrypt(C)

b'retreat'

### Try attack
Capture ciphertext $C$.

In [9]:
from secrets import randbelow

In [16]:
S = randbelow(N)
assert S < N
S

947414648544081496912840097182405403582204699678519573378494397264327793268296366503555469725894487080282777522844212274337846574680580410065132329347004943212129860698834918247067135948179183601118330361710929380182310023969576761364146625772860849395054562066073023647862779670049401231052546226678345738862062378533257596207049792756582813718496605780384496181639144818206516277256058956670253851220448299246405475364421597242987975974226780890385530032347234465622698548166564766986457686255842914425397732467228095630820913373700936906984554025309491709349771906835576752928390630157334163486179890933848025800

In [17]:
C_ = pow(S, e, N) * C % N
C_

2526468244389334837123628164441779582647088858112310058869595613950381205378870355633854444413144014071746491952829884432455661915623064118341756169516138906818635382847957319034508782219768232568696822370699266299026574656081364650287770800705477017540778128927099097263105383639217949002700279992778760455466745460710802723626293195591843939310938259520108949660084270899718162442209244233130813527281973434935347260795606519247909030996905718151655230176046387661153048460553295532252767449752796176902565039254673537922771750515179319611005527489849536681208294500406643222902802495813764323431307218987069191558

In [18]:
P_ = decrypt(C_)
P_

b"\x174\xa8\x1e\xa5\x8b\xbc\x0f\xfc_\x83\x1f!\x11'\xc0\x16$/.\x83`\x05\xf1\x00\xbfhZ\xe7\xd5\x80]\xb3\xff\xaa2\x1e\xb4\xb5\x13B\x8d\xbe\xe0\x8a\xdc\xeb\xbc\xed\xec\x08\xb7\xbc\x87\x97\xc2\x99\xc1\x11\xe9\xa3\xd4I\xbc\x86a\x17\xc3\x13VCK+T\x0f\xec\xe9l9\xf2,O\x81\x9b@\x0f2\xa0j\xa6i5\xc8\x19Mz\x9df\x00r\x05\xeb(<t\xed\x83\x84x\x0f\xa6\xafptBe_x\xa3\x8b\xaaH\x04\x86\x01xti\x89\xe6C\xbaM\x1e>M\xe1K\x98o\x1f\xbe\xd8\xca\xb7\t\xcf$\xd9q*\xaa\xb5b1%\xfe`nx\xc5k\xf0H}\xdd\x0fM\xf5\x9c\xadq\x9e\xb1\x1bo\x08\xc8\xe8\xb7\xf2\xd8\x13\xc1\x13\xcc\xdc\xf7\xf7\xa91\x94\x0f1\x16\x8eylS\xdd\xa7\xd0\x9f\x0b\xe1\xa9\xf0\tr\xd0\xb9Z\x9e#\xf3h\xe0Y\xcbX\xb2g\x8b\xe1\xd4\xdcw\xaf\xab\xd8h9X!\xe7H\x9d\xe2}m\xd6\xd7>N&0T\xae\x10\xa5\xf2[\x00m\x06\x90"

In [20]:
p = bytes_to_long(P_) * pow(S, -1, N) % N
long_to_bytes(p)

b'retreat'